# Get Online Inference from Vertex AI Endpoint

This notebook demonstrates how to get predictions from a deployed Vertex AI Endpoint.
Make sure you have authenticated with Google Cloud before running this.


In [10]:
# Import Required Libraries
from google.cloud import aiplatform

print("Libraries imported successfully.")

Libraries imported successfully.


In [11]:
# Authenticate with Google Cloud
# This will open a browser window for you to sign in
import google.auth

try:
    # Try to get default credentials
    credentials, project = google.auth.default()
    print("✅ Using existing credentials")
except:
    # If no credentials found, use gcloud auth
    print("⚠️ No credentials found. Please authenticate using one of these methods:\n")
    print("Method 1 (Recommended for local development):")
    print("   Run in terminal: gcloud auth application-default login\n")
    print("Method 2 (For Colab):")
    print("   from google.colab import auth")
    print("   auth.authenticate_user()\n")
    print("Method 3 (Service Account):")
    print("   Set environment variable: GOOGLE_APPLICATION_CREDENTIALS=path/to/key.json")

c:\Users\chamm\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\auth\_default.py:114: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


✅ Using existing credentials


## Authenticate with Google Cloud

**Important:** Run this cell first to authenticate with Google Cloud.


## 1. Configuration

Please fill in your Project ID, Region, and the Display Name of your Endpoint.


In [12]:
# --- TODO: Update these values ---
# Use the PROJECT_ID from 'gcloud projects list' command
# Based on your projects, likely one of these:
# - "dasun-487410"
# - "enterprise-cloud-architecture"
# - "project-968ac48e-8511-4620-9fe" (Chamath - Logistic Model)

PROJECT_ID = "project-968ac48e-8511-4620-9fe"  # Update this with your actual project ID
REGION = "asia-southeast1"  # Singapore region
ENDPOINT_DISPLAY_NAME = "Pipeline_Deployment_Model_Endpoint" # The name you gave your endpoint

print(f"Project: {PROJECT_ID}")
print(f"Region: {REGION}")
print(f"Endpoint Name: {ENDPOINT_DISPLAY_NAME}")

Project: project-968ac48e-8511-4620-9fe
Region: asia-southeast1
Endpoint Name: Pipeline_Deployment_Model_Endpoint


In [13]:
## 2. Initialize Vertex AI SDK
aiplatform.init(project=PROJECT_ID, location=REGION)

print("Vertex AI SDK Initialized.")

Vertex AI SDK Initialized.


In [14]:
## 3. Get Endpoint Object
# Look up the endpoint by its display name
try:
    endpoints = aiplatform.Endpoint.list(filter=f'display_name="{ENDPOINT_DISPLAY_NAME}"')

    if endpoints:
        endpoint = endpoints[0]
        print(f"✅ Found Endpoint: {endpoint.resource_name}")
    else:
        print(f"❌ Endpoint '{ENDPOINT_DISPLAY_NAME}' not found in region {REGION}.")
        print("Please check the name and region.")
except Exception as e:
    print(f"❌ Error accessing endpoint: {str(e)}")
    print("\nCommon issues:")
    print("1. Authentication: Run 'gcloud auth application-default login' in terminal")
    print("2. Project ID: Make sure you're using the project ID (name), not project number")
    print("3. Permissions: Ensure you have Vertex AI User role")
    print("4. API: Enable Vertex AI API in your project")

✅ Found Endpoint: projects/994556670658/locations/asia-southeast1/endpoints/8290495794314739712


In [15]:
# ## 4. Load Vectorizer and Prepare Data
# # Load the vectorizer that was used to train the model
# print("Loading vectorizer...")
# vectorizer = joblib.load("../07_Pipeline_Deployment/vectorizer.joblib")
# print("✅ Vectorizer loaded successfully")

# # --- THE NEW DATA ---
# # This is the new, raw text review we want to classify.
# new_review = "I recently purchased this dress and I have to say, it exceeded my expectations!"
# print(f"\nOriginal review: '{new_review}'")

# # --- PREPROCESS THE TEXT ---
# # Transform the text into numeric features using the vectorizer
# print("\nPreprocessing text...")
# sparse_matrix = vectorizer.transform([new_review])

# # Convert sparse matrix to dense array and then to list (Vertex AI expects lists)
# prediction_instances = sparse_matrix.toarray().tolist()
# print(f"✅ Text vectorized - Shape: {sparse_matrix.shape}")

# # --- MAKE THE PREDICTION CALL ---
# print("\n...Calling the endpoint...")
# response = endpoint.predict(instances=prediction_instances)

# print("\n--- Prediction Result ---")
# print(f"Prediction: {response.predictions}")

# # Interpret the result
# if response.predictions[0] == 1:
#     print("📊 Classification: POSITIVE review ✓")
# else:
#     print("📊 Classification: NEGATIVE review ✗")

## 📌 Important Note: Pipeline vs Model-Only Deployment

**ඔබ හරි!** If you deployed the **complete Pipeline** (pipeline.joblib), you should send raw text directly.

However, based on the errors we received:

- `dtype='numeric' is not compatible with arrays of bytes/strings`

This indicates the deployed model is **only the model** (model.joblib), not the pipeline.

**Two Solutions:**

1. **Current approach** (above): Load vectorizer locally and transform text before sending
2. **Better approach**: Redeploy using `pipeline.joblib` instead, then send raw text directly

If you want to try sending raw text (to test if pipeline was deployed), use the cell below:


In [16]:
# 5. TEST: Try sending raw text (if pipeline was deployed)

new_review_raw = ["I recently purchased this dress and I have to say, it exceeded my expectations!"]
print(f"Testing with raw text: '{new_review_raw[0]}'")
print("\n...Calling the endpoint with raw text...")

try:
    response = endpoint.predict(instances=new_review_raw)
    print("\n✅ SUCCESS! Pipeline was deployed - raw text works!")
    print(f"Prediction: {response.predictions}")
    if response.predictions[0] == 1:
        print("📊 Classification: POSITIVE review ✓")
    else:
        print("📊 Classification: NEGATIVE review ✗")
except Exception as e:
    print(f"\n❌ ERROR: Pipeline was NOT deployed")
    print(f"Error message: {str(e)}")
    print("\nThis confirms you need to either:")
    print("1. Use the vectorizer approach (Cell 4 above) ✓")
    print("2. Redeploy using pipeline.joblib instead of model.joblib")

Testing with raw text: 'I recently purchased this dress and I have to say, it exceeded my expectations!'

...Calling the endpoint with raw text...

✅ SUCCESS! Pipeline was deployed - raw text works!
Prediction: [1.0]
📊 Classification: POSITIVE review ✓


## 4. Get Predictions (After Pipeline Deployment)

**Use this cell AFTER deploying the complete Pipeline using the deployment notebook!**

Once you've deployed `model.joblib` (which contains the pipeline), you can send raw text directly.


In [17]:
## 4. Get Predictions with Raw Text (NO Vectorizer Needed!)

# Single review prediction
new_review = "I recently purchased this dress and I have to say, it exceeded my expectations!"
print(f"Review: '{new_review}'")
print("\n...Calling endpoint...")

response = endpoint.predict(instances=[new_review])

print("\n--- Prediction Result ---")
print(f"Raw prediction: {response.predictions[0]}")

if response.predictions[0] == 1:
    print("📊 Sentiment: POSITIVE ✅")
else:
    print("📊 Sentiment: NEGATIVE ❌")


Review: 'I recently purchased this dress and I have to say, it exceeded my expectations!'

...Calling endpoint...

--- Prediction Result ---
Raw prediction: 1.0
📊 Sentiment: POSITIVE ✅


In [18]:
## Test Multiple Reviews at Once

test_reviews = [
    "I absolutely love this dress! The fit is perfect and the material is so soft.",
    "The shirt shrunk after one wash. Very disappointed with the quality.",
    "Amazing product! Highly recommend to everyone!",
    "Terrible quality. Waste of money.",
    "It's okay, nothing special but not bad either."
]

print("Testing multiple reviews...\n")
print("=" * 80)

for i, review in enumerate(test_reviews, 1):
    response = endpoint.predict(instances=[review])
    prediction = response.predictions[0]
    sentiment = "POSITIVE ✅" if prediction == 1 else "NEGATIVE ❌"

    print(f"\n{i}. Review: {review}")
    print(f"   Prediction: {prediction} → {sentiment}")

print("\n" + "=" * 80)

Testing multiple reviews...


1. Review: I absolutely love this dress! The fit is perfect and the material is so soft.
   Prediction: 1.0 → POSITIVE ✅

2. Review: The shirt shrunk after one wash. Very disappointed with the quality.
   Prediction: -1.0 → NEGATIVE ❌

3. Review: Amazing product! Highly recommend to everyone!
   Prediction: 1.0 → POSITIVE ✅

4. Review: Terrible quality. Waste of money.
   Prediction: -1.0 → NEGATIVE ❌

5. Review: It's okay, nothing special but not bad either.
   Prediction: -1.0 → NEGATIVE ❌



---

## 🎉 Success!

**Pipeline Deployment වාසි:**

✅ **No Vectorizer needed locally** - Pipeline එකේම තියෙනවා  
✅ **Send raw text directly** - `endpoint.predict(instances=["text"])`  
✅ **Version control** - Vectorizer සහ Model එකම එක්ක update වෙනවා  
✅ **Production ready** - Clean, maintainable code

---

### 📝 Quick Reference

```python
# Single prediction
response = endpoint.predict(instances=["Your review text here"])
print(response.predictions[0])  # 1 = Positive, -1 = Negative

# Multiple predictions
reviews = ["review 1", "review 2", "review 3"]
response = endpoint.predict(instances=reviews)
print(response.predictions)  # [1, -1, 1]
```

---

### 🔗 Related Notebooks

- Training & Pipeline Creation: [07_Pipeline_Deployment/](../07_Pipeline_Deployment/)
- Deployment Guide: [Deploy_Pipeline_to_Vertex_AI.ipynb](../07_Pipeline_Deployment/Deploy_Pipeline_to_Vertex_AI.ipynb)
